In [ ]:
!pip install selenium pandas pyautogui

In [ ]:
import time

import pandas as pd
import pyautogui
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

### Set a fixed size for the Chrome Browser window when Selenium launches it.

```
chrome_options.add_argument("--window-size=1200,800")
```

### ✅ **Purpose**

*   **Consistent Layout:** Ensures that webpage elements (charts, buttons, tables) appear in predictable positions $-$ critical for interaction via Selenium and pyautogui.
*   **Avoid Hidden Elements:** Some Power BI elements may not appear correctly if the window is too small (due to the responsive layout).
*   **Stable Automation:** Fixed dimensions help avoid errors caused by varying screen sizes, especially when using pixel-based interactions.

```
pyautogui.moveTo(200, 800)
```








In [4]:
# Set up Chrome options.
chrome_options = Options()
chrome_options.add_argument("--window-size=1200,800")

driver = webdriver.Chrome(options=chrome_options)
URL = "https://app.powerbi.com/view?r=eyJrIjoiMTYxMGJkZDMtM2U4Zi00YmMyLWFhODUtYzgyMDc4OTdhOTkzIiwidCI6IjlkZTFhZmM4LTBlOTQtNDM0ZC1iOWU5LTdhZDMyNzdkMGZjYyIsImMiOjl9"
driver.get(URL)

In [5]:
chart = driver.find_element(By.CLASS_NAME, "visual-stackedAreaChart")

# Create an ActionChains object - ActionChains is a Selenium utility that simulates low-level mouse and keyboard actions.
actions = ActionChains(driver)

# Perform right-click on the element - The right-click opens a context menu in Power BI.
actions.context_click(chart).perform()

time.sleep(1)

button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@title="Show as a table"]'))
)
# button = WebDriverWait(driver, 10).until(
#     EC.element_to_be_clickable((By.XPATH, '//*[@id="0"]'))
# )

button.click()

When a Power BI visual is switched to "**Show as a table**", it doesn't load all rows into the DOM at once. Instead, it uses virtualization, rendering only the rows that are currently visible on the screen. As a result, this code will only extract a portion of data that is in view at the time it runs.

In [ ]:
table_data = driver.find_elements(By.CLASS_NAME, "scrollable-cells-container")
for i in table_data:
    print(i.text.split("\n"))

['Datum', 'Benzin', ' Dieselöl', ' Flugtreibstoffe', ' ']
['1974-02', '160,000.0', '28,000.0', '40,000.0']
['1974-03', '178,000.0', '47,000.0', '42,000.0']
['1974-04', '207,000.0', '53,000.0', '58,000.0']
['1974-05', '210,000.0', '56,000.0', '50,000.0']
['1974-06', '184,000.0', '55,000.0', '49,000.0']
['1974-07', '238,000.0', '68,000.0', '53,000.0']
['1974-08', '235,000.0', '59,000.0', '63,000.0']
['1974-09', '222,000.0', '69,000.0', '54,000.0']
['1974-10', '201,000.0', '56,000.0', '65,000.0']
['1974-11', '182,000.0', '55,000.0', '44,000.0']
['1974-12', '201,000.0', '46,000.0', '50,000.0']
['1975-01', '180,000.0', '38,000.0', '51,000.0']
['1975-02', '172,000.0', '41,000.0', '41,000.0']
['1975-03', '184,000.0', '38,000.0', '49,000.0']
['1975-04', '208,000.0', '50,000.0', '54,000.0']
['1975-05', '201,000.0', '55,000.0', '55,000.0']
['1975-06', '209,000.0', '66,000.0', '57,000.0']
['1975-07', '252,000.0', '65,000.0', '76,000.0']
['1975-08', '221,000.0', '58,000.0', '61,000.0']
['1975-09',

This snippet uses **PyAutoGUI** to programmatically move our mouse cursor to a specific screen location.

*   $moveTo(x, y)$ tells **PyAutoGUI** to glide the mouse cursor to the point $(200, 800)$.
*   **duration=1** makes that movement take 1 second (so we see a smooth slide rather than an instant jump).

In practice, this ensures that subsequent clicks or scrolls happen exactly where we need them (e.g., over a table area), by first positioning the cursor reliably at those screen coordinates.



In [ ]:
# Set target coordinates - Represent the pixel positions on our screen.
x_coordinate = 200
y_coordinate = 800

# Move the mouse.
pyautogui.moveTo(x_coordinate, y_coordinate, duration=1)

In [9]:
# Set target coordinates - Represent the pixel positions on our screen.
x_coordinate = 200
y_coordinate = 800

# Make sure the mouse is hovering over the table's scrollable region so that subsequent scroll events affect the right element.
pyautogui.moveTo(x_coordinate, y_coordinate, duration=1)

# all_rows: This will accumulate every row's data as a list of lists.
# last_row: Holds the content of the final row.
# count: A simple loop counter (useful for debugging or logging).
all_rows = []
last_row = None
count = 0

# Loop to Scroll & Scrape until the bottom.
while True:
    # Grab whatever rows are currently rendered.
    table_data = driver.find_elements(By.CLASS_NAME, "scrollable-cells-container")

    # Extract all the visible rows from the Power BI table at this moment (skip the header at index 0).
    for row in table_data[1:]:
        row_data = row.text.split("\n")
        all_rows.append(row_data)

    # If the last row hasn't changed, we assume we've reached the bottom of the table.
    current_last_row = table_data[-1].text.split("\n") if table_data else None
    if current_last_row == last_row:
        break

    last_row = current_last_row

    # scroll(-300): Scroll the window down by 300 "clicks".
    # This nudges the table so that its next batch of rows comes into view and gets injected into the DOM.
    pyautogui.scroll(-300)

    # time.sleep(0.3): Small pause to let those new rows render before the next iteration.
    time.sleep(0.3)

    # Increment the count to track how many iterations we've gone through.
    count += 1

driver.quit()

columns = ["date", "benzin", "dieselol", "flugtreibstoffe"]

# Create a DataFrame from the collected rows.
df = pd.DataFrame(all_rows, columns=columns)
df.drop_duplicates(inplace=True, ignore_index=True)

print("DataFrame Length:", len(df))
df

DataFrame Length: 614


,date,benzin,dieselol,flugtreibstoffe
0,1974-02,"160,000.0","28,000.0","40,000.0"
1,1974-03,"178,000.0","47,000.0","42,000.0"
2,1974-04,"207,000.0","53,000.0","58,000.0"
3,1974-05,"210,000.0","56,000.0","50,000.0"
4,1974-06,"184,000.0","55,000.0","49,000.0"
...,...,...,...,...
609,2024-11,"175,000.0","225,000.0","123,000.0"
610,2024-12,"181,000.0","214,000.0","158,000.0"
611,2025-01,"174,000.0","188,000.0","128,000.0"
612,2025-02,"160,000.0","197,000.0","141,000.0"
